In [1]:
!pip install polars


In [1]:
import polars as pl
import glob
import pandas as pd

In [5]:
# polars settings
#show 50 columns by default
pl.Config().set_tbl_cols(50)

polars.config.Config

In [ ]:
# change display of result container here:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 150%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
# get all the filenames
csv_files_non_germans = glob.glob(r"C:\Users\Me\migration and crime data\crime\*nichtdeutsch*.csv")
csv_files_germans = [x for x in glob.glob(r"C:\Users\Me\migration and crime data\crime\*deutsch*.csv") if x not in csv_files_non_germans]

In [3]:
# check if the german list is missing files
csv_files_germans

['C:\\Users\\Me\\migration and crime data\\crime\\PKS2018-BKA-LKS-TV-09-T40-Kreise-TV-deutsch_csv.csv',
 'C:\\Users\\Me\\migration and crime data\\crime\\PKS2019-KR-TV-03-T40-Kreise-TV-deutsch_csv.csv',
 'C:\\Users\\Me\\migration and crime data\\crime\\PKS2021-KR-TV-03-T40-Kreise-TV-deutsch_csv.csv',
 'C:\\Users\\Me\\migration and crime data\\crime\\PKS2023-KR-TV-03-T40-Kreise-TV-deutsch_csv.csv']

In [4]:
# check if the non german list is missing files
csv_files_non_germans

['C:\\Users\\Me\\migration and crime data\\crime\\PKS2018-BKA-LKS-TV-11-T50-Kreise-TV-nichtdeutsch_csv.csv',
 'C:\\Users\\Me\\migration and crime data\\crime\\PKS2019-KR-TV-05-T50-Kreise-TV-nichtdeutsch_csv.csv',
 'C:\\Users\\Me\\migration and crime data\\crime\\PKS2021-KR-TV-05-T50-Kreise-TV-nichtdeutsch_csv.csv',
 'C:\\Users\\Me\\migration and crime data\\crime\\PKS203-KR-TV-05-T50-Kreise-TV-nichtdeutsch_csv.csv']

In [4]:
# detect encoding
import chardet
with open('C:\\Users\\Me\\migration and crime data\\crime\\PKS2018-BKA-LKS-TV-09-T40-Kreise-TV-deutsch_csv.csv', 'rb') as f:
    result = chardet.detect(f.read())

In [5]:
# check file content (first 5 lines)
with open('C:\\Users\\Me\\migration and crime data\\crime\\PKS2018-BKA-LKS-TV-09-T40-Kreise-TV-deutsch_csv.csv', "r", encoding=result['encoding']) as f:
    lines = f.readlines()[:5]  # Get first 5 lines
    for i, line in enumerate(lines, 1):
         print(f"Line {i}: {line.strip()}")

Line 1: 1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27
Line 2: Schluessel;Straftat;Gemeindeschluessel;Stadt/Landkreis;Kreisart;Sexus;Tatverdaechtige deutsch insgesamt - Anzahl;Kinder bis unter 6 Jahre - Anzahl;Kinder 6 bis unter 8 Jahre Anzahl;Kinder 8 bis unter 10 Jahre - Anzahl;Kinder 10 bis unter 12 Jahre - Anzahl;Kinder 12 bis unter 14 Jahre - Anzahl;Kinder bis unter 14 Jahre insgesamt (Anzahl Spalte 8 bis 12);Jugendliche 14 bis unter 16 Jahre - Anzahl;Jugendliche 16 bis unter 18 Jahre - Anzahl;Jugendliche 14 bis unter 18 Jahre insgesamt (Anzahl Spalte 14 und 15);Heranwachsende 18 bis unter 21 Jahre - Anzahl;Tatverdaechtige insgesamt unter 21 Jahre (Anzahl Spalte 13, 16 und 17);Erwachsene 21 bis unter 23 Jahre - Anzahl;Erwachsene 23 bis unter 25 Jahre - Anzahl;Erwachsene 21 bis unter 25 Jahre (Anzahl Spalte 19 und 20);Erwachsene 25 bis unter 30 Jahre;Erwachsene 30 bis unter 40 Jahre;Erwachsene 40 bis unter 50 Jahre;Erwachsene 50 bis unter 60 Jahre;Erwachsen

In [23]:
# when reading the data define a schema
enforced_schema = {
    # Identifier columns
    "Schluessel": pl.String,
    "Gemeindeschluessel": pl.String,
    
    # Categorical columns
    "Straftat": pl.Categorical,
    "Stadt/Landkreis": pl.Categorical,
    "Kreisart": pl.Categorical,
    "Sexus": pl.Categorical,
    
    # Numeric columns with German formatting (1.204 -> 1204)
    **{
        col: pl.Int64
        for col in [
            "Tatverdaechtige insgesamt - Anzahl",
            "Kinder bis unter 6 Jahre - Anzahl",
            "Kinder 6 bis unter 8 Jahre Anzahl",
            "Kinder 8 bis unter 10 Jahre - Anzahl",
            "Kinder 10 bis unter 12 Jahre - Anzahl",
            "Kinder 12 bis unter 14 Jahre - Anzahl",
            "Kinder bis unter 14 Jahre insgesamt (Anzahl Spalte 8 bis 12)",
            "Jugendliche 14 bis unter 16 Jahre - Anzahl",
            "Jugendliche 16 bis unter 18 Jahre - Anzahl",
            "Jugendliche 14 bis unter 18 Jahre insgesamt (Anzahl Spalte 14 und 15)",
            "Heranwachsende 18 bis unter 21 Jahre - Anzahl",
            "Tatverdaechtige insgesamt unter 21 Jahre (Anzahl Spalte 13, 16 und 17)",
            "Erwachsene 21 bis unter 23 Jahre - Anzahl",
            "Erwachsene 23 bis unter 25 Jahre - Anzahl",
            "Erwachsene 21 bis unter 25 Jahre (Anzahl Spalte 19 und 20)",
            "Erwachsene 25 bis unter 30 Jahre",
            "Erwachsene 30 bis unter 40 Jahre",
            "Erwachsene 40 bis unter 50 Jahre",
            "Erwachsene 50 bis unter 60 Jahre",
            "Erwachsene 60 Jahre und aelter",
            "Erwachsene ab 21 Jahre (Anzahl Spalten 21 bis 26)"
        ]
    }
}





In [27]:
# read the data into polars data frames: use the encoding found above, skip the first row (only numbers), apply schema defined above and replace umlaute from column name
dataframes_non_germans = [pl.read_csv(file, encoding = result['encoding'], skip_rows=1,ignore_errors=True, has_header=False,schema=enforced_schema,
   ).with_columns(pl.col("Stadt/Landkreis").cast(pl.String).replace("ü", "ue").str.to_uppercase(),pl.col("Kreisart").cast(pl.String).replace("KfS", "Kreisfreie Stadt")) for file in csv_files_non_germans]
# now for the german data set
dataframes_germans = [pl.read_csv(file, encoding = result['encoding'], skip_rows=1,ignore_errors=True, has_header=False,schema=enforced_schema,
    ).with_columns(pl.col("Stadt/Landkreis").cast(pl.String).replace("ü", "ue").str.to_uppercase(),pl.col("Kreisart").cast(pl.String).replace("KfS", "Kreisfreie Stadt")) for file in csv_files_non_germans]

In [15]:
# look at the columns
dataframes_germans[0].columns

['Schluessel',
 'Straftat',
 'Gemeindeschluessel',
 'Stadt/Landkreis',
 'Kreisart',
 'Sexus',
 'Tatverdaechtige deutsch insgesamt - Anzahl',
 'Kinder bis unter 6 Jahre - Anzahl',
 'Kinder 6 bis unter 8 Jahre Anzahl',
 'Kinder 8 bis unter 10 Jahre - Anzahl',
 'Kinder 10 bis unter 12 Jahre - Anzahl',
 'Kinder 12 bis unter 14 Jahre - Anzahl',
 'Kinder bis unter 14 Jahre insgesamt (Anzahl Spalte 8 bis 12)',
 'Jugendliche 14 bis unter 16 Jahre - Anzahl',
 'Jugendliche 16 bis unter 18 Jahre - Anzahl',
 'Jugendliche 14 bis unter 18 Jahre insgesamt (Anzahl Spalte 14 und 15)',
 'Heranwachsende 18 bis unter 21 Jahre - Anzahl',
 'Tatverdaechtige insgesamt unter 21 Jahre (Anzahl Spalte 13, 16 und 17)',
 'Erwachsene 21 bis unter 23 Jahre - Anzahl',
 'Erwachsene 23 bis unter 25 Jahre - Anzahl',
 'Erwachsene 21 bis unter 25 Jahre (Anzahl Spalte 19 und 20)',
 'Erwachsene 25 bis unter 30 Jahre',
 'Erwachsene 30 bis unter 40 Jahre',
 'Erwachsene 40 bis unter 50 Jahre',
 'Erwachsene 50 bis unter 60 Jahre

In [16]:
# combine all the data frames into one data frame. 
# One column changes name(Tatverdaechtige insgesamt - Anzahl) this is why diagonally-relaxed is chosen
combined_df_all_years_germans = pl.concat(dataframes_germans, how="diagonal_relaxed")

In [18]:
# look at the data
combined_df_all_years_germans

Schluessel,Straftat,Gemeindeschluessel,Stadt/Landkreis,Kreisart,Sexus,Tatverdaechtige deutsch insgesamt - Anzahl,Kinder bis unter 6 Jahre - Anzahl,Kinder 6 bis unter 8 Jahre Anzahl,Kinder 8 bis unter 10 Jahre - Anzahl,Kinder 10 bis unter 12 Jahre - Anzahl,Kinder 12 bis unter 14 Jahre - Anzahl,Kinder bis unter 14 Jahre insgesamt (Anzahl Spalte 8 bis 12),Jugendliche 14 bis unter 16 Jahre - Anzahl,Jugendliche 16 bis unter 18 Jahre - Anzahl,Jugendliche 14 bis unter 18 Jahre insgesamt (Anzahl Spalte 14 und 15),Heranwachsende 18 bis unter 21 Jahre - Anzahl,"Tatverdaechtige insgesamt unter 21 Jahre (Anzahl Spalte 13, 16 und 17)",Erwachsene 21 bis unter 23 Jahre - Anzahl,Erwachsene 23 bis unter 25 Jahre - Anzahl,Erwachsene 21 bis unter 25 Jahre (Anzahl Spalte 19 und 20),Erwachsene 25 bis unter 30 Jahre,Erwachsene 30 bis unter 40 Jahre,Erwachsene 40 bis unter 50 Jahre,Erwachsene 50 bis unter 60 Jahre,Erwachsene 60 Jahre und aelter,Erwachsene ab 21 Jahre (Anzahl Spalten 21 bis 26),Tatverdaechtige insgesamt - Anzahl
str,str,i64,str,str,str,str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""------""","""Straftaten insgesamt""",1001,"""Flensburg""","""KfS""","""M""","""1,652""",0,4,10,13,28,"""55""","""59""","""74""","""133""","""139""","""327""","""115""","""107""","""222""","""226""","""372""","""235""","""168""","""102""","""1,325""",null
"""------""","""Straftaten insgesamt""",1002,"""Kiel""","""KfS""","""M""","""4,215""",1,12,15,29,61,"""118""","""177""","""184""","""361""","""331""","""810""","""279""","""246""","""525""","""599""","""844""","""611""","""516""","""310""","""3,405""",null
"""------""","""Straftaten insgesamt""",1003,"""Lübeck""","""KfS""","""M""","""4,262""",2,13,18,22,53,"""108""","""144""","""227""","""371""","""315""","""794""","""246""","""251""","""497""","""536""","""903""","""636""","""560""","""336""","""3,468""",null
"""------""","""Straftaten insgesamt""",1004,"""Neumünster""","""KfS""","""M""","""1,882""",1,3,6,10,32,"""52""","""64""","""104""","""168""","""172""","""392""","""111""","""103""","""214""","""262""","""373""","""267""","""212""","""162""","""1,490""",null
"""------""","""Straftaten insgesamt""",1051,"""Dithmarschen""","""K""","""M""","""1,921""",3,3,7,23,40,"""76""","""95""","""107""","""202""","""188""","""466""","""105""","""112""","""217""","""232""","""332""","""245""","""237""","""192""","""1,455""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""972500""","""Unerlaubt eingereiste/aufhälti…",16073,"""Saalfeld-Rudolstadt""","""LK""","""X""",null,0,0,0,0,0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
"""972500""","""Unerlaubt eingereiste/aufhälti…",16074,"""Saale-Holzland-Kreis""","""LK""","""X""",null,0,0,0,0,0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
"""972500""","""Unerlaubt eingereiste/aufhälti…",16075,"""Saale-Orla-Kreis""","""LK""","""X""",null,0,0,0,0,0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
